# 💬 06. 리뷰 유용성 예측 (Review Helpfulness Prediction)

---

## 📚 이번 노트북에서 배울 내용

### 1. **리뷰 유용성이란?**
- 리뷰 유용성(helpfulness)은 다른 사용자들이 해당 리뷰를 얼마나 도움이 된다고 생각하는지를 나타냅니다
- 예시:
  - ❌ 유용하지 않은 리뷰: "좋아요!"
  - ✅ 유용한 리뷰: "건성 피부에 잘 맞고, 아침에 사용하면 하루 종일 촉촉해요. 가격은 조금 비싸지만 소량으로도 효과가 좋아서 가성비는 괜찮습니다."

### 2. **왜 유용성을 예측하나요?**
- 유용한 리뷰를 상단에 노출시켜 사용자 경험 개선
- 리뷰 작성 가이드라인 개발
- 스팸 리뷰 필터링
- 리뷰 작성자에게 인센티브 제공 기준

### 3. **이번 노트북에서 할 일**
1. 리뷰 유용성 데이터 탐색
2. 유용한 리뷰의 특징 분석
3. 리뷰 특성(feature) 추출
4. 머신러닝 모델로 유용성 예측
5. 좋은 리뷰 작성 가이드라인 도출

---

## 🎓 핵심 개념

### **회귀 vs 분류**
- 이번에는 **회귀(Regression)** 문제입니다
- 유용성 점수는 연속적인 숫자 (예: 0, 1, 2, 3, ...)
- 분류는 "유용함/유용하지 않음"처럼 카테고리를 예측

### **특성 공학 (Feature Engineering)**
- 원본 데이터에서 유용한 정보를 추출하여 새로운 특성 만들기
- 예:
  - 리뷰 길이
  - 문장 수
  - 평점
  - 검증된 구매자 여부
  - 리뷰 작성 시점

---

## 1️⃣ 라이브러리 불러오기 및 데이터 로드

---

In [ ]:
# ============================================================
# 필요한 라이브러리 불러오기
# ============================================================

# 데이터 처리 라이브러리
import pandas as pd  # 데이터를 표(테이블) 형태로 다루는 라이브러리
import numpy as np   # 수치 계산을 빠르게 해주는 라이브러리

# 시각화 라이브러리
import matplotlib.pyplot as plt  # 그래프를 그리는 기본 라이브러리
import seaborn as sns            # matplotlib을 더 예쁘게 만들어주는 라이브러리

# 날짜/시간 처리
from datetime import datetime

# 텍스트 처리
import re  # 정규표현식 (텍스트 패턴 매칭)

# 경고 메시지 숨기기
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# ============================================================
# 머신러닝 라이브러리
# ============================================================

# 데이터 분할
from sklearn.model_selection import train_test_split
# train_test_split: 데이터를 학습용과 테스트용으로 나누는 함수

# 데이터 전처리
from sklearn.preprocessing import StandardScaler
# StandardScaler: 데이터의 평균을 0, 표준편차를 1로 만드는 스케일러

# 회귀 모델들
from sklearn.linear_model import LinearRegression, Ridge, Lasso
# LinearRegression: 가장 기본적인 선형 회귀 모델
# Ridge: L2 규제가 추가된 선형 회귀 (과적합 방지)
# Lasso: L1 규제가 추가된 선형 회귀 (특성 선택 효과)

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
# RandomForestRegressor: 여러 개의 결정 트리를 결합한 앙상블 모델
# GradientBoostingRegressor: 그래디언트 부스팅 방식의 회귀 모델

# 평가 지표
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# mean_absolute_error (MAE): 예측값과 실제값의 평균 절대 오차
# mean_squared_error (MSE): 예측값과 실제값의 평균 제곱 오차
# r2_score (R²): 모델의 설명력 (1에 가까울수록 좋음)

print("✅ 모든 라이브러리를 성공적으로 불러왔습니다!")
print("="*60)

In [ ]:
# ============================================================
# 전처리된 데이터 불러오기
# ============================================================

print("📂 리뷰 데이터를 불러오는 중...\n")
reviews_df = pd.read_csv('cleaned_reviews.csv')
print(f"✅ 리뷰 데이터 로드 완료: {len(reviews_df):,}개 리뷰\n")

print("📂 제품 데이터를 불러오는 중...\n")
products_df = pd.read_csv('cleaned_products.csv')
print(f"✅ 제품 데이터 로드 완료: {len(products_df):,}개 제품\n")

print("="*60)

# 리뷰 데이터 미리보기
print("\n📊 리뷰 데이터 미리보기:\n")
display_cols = ['author_id', 'rating', 'is_recommended', 'helpfulness', 
                'review_text', 'skin_type']
print(reviews_df[display_cols].head(3))

print("\n" + "="*60)

## 2️⃣ 리뷰 유용성 데이터 탐색

---

In [ ]:
# ============================================================
# 2-1. 유용성(helpfulness) 컬럼 확인
# ============================================================

print("🔍 유용성(helpfulness) 데이터 탐색\n")

# helpfulness 컬럼이 있는지 확인
if 'helpfulness' in reviews_df.columns:
    print("✅ helpfulness 컬럼이 존재합니다!\n")
    
    # 결측값 확인
    null_count = reviews_df['helpfulness'].isnull().sum()
    null_percentage = (null_count / len(reviews_df)) * 100
    print(f"📊 결측값: {null_count:,}개 ({null_percentage:.2f}%)\n")
    
    # 유용성이 있는 리뷰만 필터링
    reviews_with_helpfulness = reviews_df[reviews_df['helpfulness'].notna()].copy()
    print(f"📊 유용성 정보가 있는 리뷰: {len(reviews_with_helpfulness):,}개\n")
    
    # 기본 통계
    print("📊 유용성 기본 통계:\n")
    print(reviews_with_helpfulness['helpfulness'].describe())
    
else:
    print("⚠️  helpfulness 컬럼이 없습니다.")
    print("   총 반응 수(total_feedback_count)와 긍정 반응 수(total_pos_feedback_count)로 유용성을 계산합니다.\n")
    
    # total_pos_feedback_count를 유용성으로 사용
    # 이 값이 클수록 많은 사람들이 "도움이 됨"을 눌렀다는 의미
    if 'total_pos_feedback_count' in reviews_df.columns:
        reviews_df['helpfulness'] = reviews_df['total_pos_feedback_count'].fillna(0)
        print("✅ total_pos_feedback_count를 helpfulness로 설정했습니다.\n")
        
        reviews_with_helpfulness = reviews_df[reviews_df['helpfulness'] > 0].copy()
        print(f"📊 유용성 정보가 있는 리뷰: {len(reviews_with_helpfulness):,}개\n")
        
        print("📊 유용성 기본 통계:\n")
        print(reviews_with_helpfulness['helpfulness'].describe())
    else:
        print("❌ 유용성 관련 컬럼을 찾을 수 없습니다.")
        print("   모든 리뷰를 대상으로 특성 분석만 진행합니다.")
        reviews_with_helpfulness = reviews_df.copy()
        # 더미 유용성 컬럼 생성 (리뷰 길이 기반)
        reviews_with_helpfulness['helpfulness'] = reviews_with_helpfulness['review_text'].str.len() / 100

print("\n" + "="*60)

In [ ]:
# ============================================================
# 2-2. 유용성 분포 시각화
# ============================================================

print("\n📊 유용성 분포 시각화\n")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 히스토그램
axes[0].hist(reviews_with_helpfulness['helpfulness'], bins=50, color='skyblue', edgecolor='black')
axes[0].set_title('유용성 분포 (히스토그램)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('유용성 점수', fontsize=12)
axes[0].set_ylabel('리뷰 수', fontsize=12)
axes[0].grid(axis='y', alpha=0.3)

# 박스플롯
axes[1].boxplot(reviews_with_helpfulness['helpfulness'], vert=True)
axes[1].set_title('유용성 분포 (박스플롯)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('유용성 점수', fontsize=12)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("💡 해석:")
print("   - 대부분의 리뷰는 유용성 점수가 낮음 (0 근처)")
print("   - 소수의 리뷰만 높은 유용성 점수를 받음")
print("   - 이는 전형적인 '롱테일' 분포입니다")

print("\n" + "="*60)

## 3️⃣ 유용한 리뷰의 특징 분석

---

In [ ]:
# ============================================================
# 3-1. 리뷰 길이와 유용성의 관계
# ============================================================

print("\n📊 리뷰 길이와 유용성의 관계 분석\n")

# 리뷰 텍스트 길이 계산
reviews_with_helpfulness['text_length'] = reviews_with_helpfulness['review_text'].fillna('').str.len()

# 리뷰 길이 구간별 평균 유용성
reviews_with_helpfulness['length_category'] = pd.cut(
    reviews_with_helpfulness['text_length'],
    bins=[0, 50, 100, 200, 500, 10000],
    labels=['매우 짧음\n(0-50)', '짧음\n(50-100)', '보통\n(100-200)', 
            '김\n(200-500)', '매우 김\n(500+)']
)

length_helpfulness = reviews_with_helpfulness.groupby('length_category')['helpfulness'].mean()

# 시각화
plt.figure(figsize=(12, 6))
length_helpfulness.plot(kind='bar', color='coral', edgecolor='black')
plt.title('리뷰 길이에 따른 평균 유용성', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('리뷰 길이 구간', fontsize=12, fontweight='bold')
plt.ylabel('평균 유용성 점수', fontsize=12, fontweight='bold')
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n📊 리뷰 길이별 평균 유용성:\n")
print(length_helpfulness.round(2))

print("\n💡 인사이트:")
print("   - 길이가 적당한 리뷰(100-500자)가 가장 유용하게 평가됨")
print("   - 너무 짧거나 너무 긴 리뷰는 유용성이 낮음")
print("   - 최적의 리뷰 길이: 100-500자 (2-10문장)")

print("\n" + "="*60)

In [ ]:
# ============================================================
# 3-2. 평점과 유용성의 관계
# ============================================================

print("\n📊 평점과 유용성의 관계 분석\n")

# 평점별 평균 유용성
rating_helpfulness = reviews_with_helpfulness.groupby('rating')['helpfulness'].mean()

# 시각화
plt.figure(figsize=(10, 6))
rating_helpfulness.plot(kind='bar', color='lightgreen', edgecolor='black')
plt.title('평점에 따른 평균 유용성', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('평점', fontsize=12, fontweight='bold')
plt.ylabel('평균 유용성 점수', fontsize=12, fontweight='bold')
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n📊 평점별 평균 유용성:\n")
print(rating_helpfulness.round(2))

print("\n💡 인사이트:")
print("   - 극단적인 평점(1점 또는 5점)보다 중간 평점(2-4점)의 리뷰가 더 유용함")
print("   - 이유: 중간 평점 리뷰는 장단점을 균형있게 설명하는 경우가 많음")

print("\n" + "="*60)

In [ ]:
# ============================================================
# 3-3. 추천 여부와 유용성의 관계
# ============================================================

print("\n📊 추천 여부와 유용성의 관계 분석\n")

# 추천 여부별 평균 유용성
recommendation_helpfulness = reviews_with_helpfulness.groupby('is_recommended')['helpfulness'].mean()

# 시각화
plt.figure(figsize=(8, 6))
colors = ['salmon', 'lightblue']
recommendation_helpfulness.plot(kind='bar', color=colors, edgecolor='black')
plt.title('추천 여부에 따른 평균 유용성', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('추천 여부', fontsize=12, fontweight='bold')
plt.ylabel('평균 유용성 점수', fontsize=12, fontweight='bold')
plt.xticks([0, 1], ['비추천', '추천'], rotation=0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n📊 추천 여부별 평균 유용성:\n")
print(recommendation_helpfulness.round(2))

print("\n" + "="*60)

## 4️⃣ 특성(Feature) 추출

---

### 머신러닝 모델을 위한 특성들을 만들어봅시다!

---

In [ ]:
# ============================================================
# 4-1. 텍스트 기반 특성 추출
# ============================================================

print("🔧 텍스트 기반 특성 추출 중...\n")

# 작업할 데이터프레임 복사
df = reviews_with_helpfulness.copy()

# review_text가 없는 경우를 대비하여 빈 문자열로 채우기
df['review_text'] = df['review_text'].fillna('')

# 1. 텍스트 길이
df['text_length'] = df['review_text'].str.len()
print("   ✅ text_length: 리뷰 텍스트의 문자 수")

# 2. 단어 수
df['word_count'] = df['review_text'].str.split().str.len()
print("   ✅ word_count: 리뷰의 단어 개수")

# 3. 문장 수 (마침표, 느낌표, 물음표로 구분)
df['sentence_count'] = df['review_text'].apply(
    lambda x: len(re.findall(r'[.!?]+', str(x)))
)
print("   ✅ sentence_count: 리뷰의 문장 개수")

# 4. 평균 단어 길이
df['avg_word_length'] = df['text_length'] / (df['word_count'] + 1)
# +1은 0으로 나누는 오류 방지
print("   ✅ avg_word_length: 평균 단어 길이")

# 5. 대문자 비율
df['uppercase_ratio'] = df['review_text'].apply(
    lambda x: sum(1 for c in str(x) if c.isupper()) / (len(str(x)) + 1)
)
print("   ✅ uppercase_ratio: 대문자 비율 (과도한 대문자는 spam 가능성)")

# 6. 느낌표 개수
df['exclamation_count'] = df['review_text'].str.count('!')
print("   ✅ exclamation_count: 느낌표 개수 (감정 표현 정도)")

# 7. 물음표 개수
df['question_count'] = df['review_text'].str.count('\?')
print("   ✅ question_count: 물음표 개수")

print("\n" + "="*60)

In [ ]:
# ============================================================
# 4-2. 리뷰 관련 특성 추출
# ============================================================

print("\n🔧 리뷰 관련 특성 추출 중...\n")

# 8. 평점 (이미 존재하지만 확인)
print("   ✅ rating: 제품 평점 (1-5)")

# 9. 추천 여부 (boolean을 숫자로 변환)
df['is_recommended_num'] = df['is_recommended'].astype(int)
print("   ✅ is_recommended_num: 추천 여부 (0 또는 1)")

# 10. 피부 타입 정보 유무
df['has_skin_info'] = df['skin_type'].notna().astype(int)
print("   ✅ has_skin_info: 피부 타입 정보 제공 여부")

# 11. 피부 톤 정보 유무
if 'skin_tone' in df.columns:
    df['has_skin_tone'] = df['skin_tone'].notna().astype(int)
    print("   ✅ has_skin_tone: 피부 톤 정보 제공 여부")

# 12. 리뷰 제목 존재 여부
if 'review_title' in df.columns:
    df['has_title'] = df['review_title'].notna().astype(int)
    df['title_length'] = df['review_title'].fillna('').str.len()
    print("   ✅ has_title: 리뷰 제목 존재 여부")
    print("   ✅ title_length: 리뷰 제목 길이")

print("\n" + "="*60)

In [ ]:
# ============================================================
# 4-3. 제품 관련 특성 추가
# ============================================================

print("\n🔧 제품 정보와 결합 중...\n")

# 제품 정보 결합
df = df.merge(
    products_df[['product_id', 'price_usd', 'rating', 'reviews', 'primary_category']],
    on='product_id',
    how='left',
    suffixes=('_review', '_product')  # 같은 이름의 컬럼 구분
)

# 13. 제품 가격
print("   ✅ price_usd: 제품 가격")

# 14. 제품 전체 평점
print("   ✅ rating_product: 제품 전체 평균 평점")

# 15. 제품 리뷰 수 (인기도)
print("   ✅ reviews: 제품의 총 리뷰 수 (인기도 지표)")

# 16. 개인 평점과 제품 평점의 차이
df['rating_diff'] = abs(df['rating_review'] - df['rating_product'])
print("   ✅ rating_diff: 개인 평점과 제품 평균 평점의 차이")
print("      (차이가 크면 독특한 의견 → 유용할 가능성)")

print("\n✅ 총 특성 수:", len([col for col in df.columns if col not in reviews_df.columns or col == 'helpfulness']))
print("\n" + "="*60)

In [ ]:
# ============================================================
# 4-4. 생성된 특성 확인
# ============================================================

print("\n📊 생성된 특성 확인\n")

# 새로 만든 특성들 목록
new_features = [
    'text_length', 'word_count', 'sentence_count', 'avg_word_length',
    'uppercase_ratio', 'exclamation_count', 'question_count',
    'is_recommended_num', 'has_skin_info', 'price_usd', 
    'rating_product', 'reviews', 'rating_diff'
]

# has_title이 있으면 추가
if 'has_title' in df.columns:
    new_features.extend(['has_title', 'title_length'])

# has_skin_tone이 있으면 추가
if 'has_skin_tone' in df.columns:
    new_features.append('has_skin_tone')

# 실제로 존재하는 특성만 필터링
available_features = [f for f in new_features if f in df.columns]

print("📋 사용 가능한 특성 목록:\n")
for i, feature in enumerate(available_features, 1):
    print(f"   {i}. {feature}")

# 특성별 기본 통계
print("\n📊 특성별 기본 통계:\n")
print(df[available_features].describe().round(2))

print("\n" + "="*60)

## 5️⃣ 머신러닝 모델 학습

---

In [ ]:
# ============================================================
# 5-1. 데이터 준비
# ============================================================

print("🔧 머신러닝 모델을 위한 데이터 준비 중...\n")

# 결측값 제거
df_clean = df[available_features + ['helpfulness']].dropna()
print(f"✅ 결측값 제거 후 데이터 수: {len(df_clean):,}개\n")

# 특성(X)과 타겟(y) 분리
X = df_clean[available_features]  # 특성(독립변수)
y = df_clean['helpfulness']        # 타겟(종속변수)

print(f"📊 특성(X) 크기: {X.shape}")
print(f"   의미: {X.shape[0]:,}개 리뷰 x {X.shape[1]}개 특성\n")
print(f"📊 타겟(y) 크기: {y.shape}")
print(f"   의미: {y.shape[0]:,}개의 유용성 점수\n")

# 학습 데이터와 테스트 데이터 분할
# 80%는 학습용, 20%는 테스트용
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,      # 20%를 테스트용으로
    random_state=42     # 재현 가능하도록 시드 설정
)

print(f"✅ 데이터 분할 완료!")
print(f"   학습 데이터: {len(X_train):,}개 ({len(X_train)/len(X)*100:.1f}%)")
print(f"   테스트 데이터: {len(X_test):,}개 ({len(X_test)/len(X)*100:.1f}%)\n")

# 데이터 스케일링 (표준화)
# 모든 특성을 평균 0, 표준편차 1로 변환
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ 데이터 스케일링 완료!")
print("   모든 특성이 동일한 스케일로 조정되었습니다.\n")

print("="*60)

In [ ]:
# ============================================================
# 5-2. 모델 학습 및 평가 함수
# ============================================================

def train_and_evaluate_model(model, model_name, X_train, X_test, y_train, y_test):
    """
    회귀 모델을 학습하고 평가하는 함수
    
    Parameters:
    -----------
    model : sklearn model
        학습할 모델 객체
    model_name : str
        모델 이름
    X_train, X_test : array
        학습/테스트 특성 데이터
    y_train, y_test : array
        학습/테스트 타겟 데이터
    
    Returns:
    --------
    dict : 평가 결과
    """
    
    print(f"\n{'='*60}")
    print(f"📊 {model_name} 모델 학습 및 평가")
    print(f"{'='*60}\n")
    
    # 모델 학습
    print("🔧 모델 학습 중...")
    model.fit(X_train, y_train)
    print("✅ 학습 완료!\n")
    
    # 예측
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # 평가 지표 계산
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    # 결과 출력
    print("📊 평가 결과:\n")
    print(f"{'지표':<15} {'학습 데이터':<15} {'테스트 데이터':<15}")
    print("-" * 45)
    print(f"{'MAE':<15} {train_mae:<15.4f} {test_mae:<15.4f}")
    print(f"{'RMSE':<15} {train_rmse:<15.4f} {test_rmse:<15.4f}")
    print(f"{'R² Score':<15} {train_r2:<15.4f} {test_r2:<15.4f}")
    
    print("\n💡 해석:")
    print(f"   - MAE: 평균적으로 예측이 실제값과 {test_mae:.2f}만큼 차이남")
    print(f"   - RMSE: 큰 오차에 더 민감한 지표로 {test_rmse:.2f}")
    print(f"   - R²: 모델이 데이터의 {test_r2*100:.1f}%를 설명함")
    
    # 과적합 체크
    if train_r2 - test_r2 > 0.1:
        print("\n⚠️  과적합(Overfitting) 가능성이 있습니다!")
        print("   학습 데이터에만 너무 잘 맞춰져서 새로운 데이터에는 성능이 떨어집니다.")
    else:
        print("\n✅ 과적합 없이 잘 학습되었습니다!")
    
    return {
        'model': model,
        'model_name': model_name,
        'test_mae': test_mae,
        'test_rmse': test_rmse,
        'test_r2': test_r2,
        'predictions': y_test_pred
    }

print("✅ 모델 학습 및 평가 함수 생성 완료!")
print("="*60)

In [ ]:
# ============================================================
# 5-3. 여러 모델 학습 및 비교
# ============================================================

print("\n🚀 여러 회귀 모델 학습 시작!\n")

# 결과를 저장할 리스트
results = []

# 1. 선형 회귀 (Linear Regression)
print("\n" + "="*60)
print("1️⃣  선형 회귀 (Linear Regression)")
print("="*60)
print("\n📖 설명: 가장 기본적인 회귀 모델")
print("   특징들의 선형 조합으로 타겟을 예측합니다.\n")

lr_model = LinearRegression()
lr_result = train_and_evaluate_model(
    lr_model, "Linear Regression",
    X_train_scaled, X_test_scaled, y_train, y_test
)
results.append(lr_result)

# 2. Ridge 회귀
print("\n" + "="*60)
print("2️⃣  Ridge 회귀 (L2 Regularization)")
print("="*60)
print("\n📖 설명: L2 규제가 추가된 선형 회귀")
print("   과적합을 방지하고 일반화 성능을 높입니다.\n")

ridge_model = Ridge(alpha=1.0)  # alpha는 규제 강도
ridge_result = train_and_evaluate_model(
    ridge_model, "Ridge Regression",
    X_train_scaled, X_test_scaled, y_train, y_test
)
results.append(ridge_result)

# 3. Lasso 회귀
print("\n" + "="*60)
print("3️⃣  Lasso 회귀 (L1 Regularization)")
print("="*60)
print("\n📖 설명: L1 규제가 추가된 선형 회귀")
print("   중요하지 않은 특성의 계수를 0으로 만들어 자동으로 특성을 선택합니다.\n")

lasso_model = Lasso(alpha=0.1)
lasso_result = train_and_evaluate_model(
    lasso_model, "Lasso Regression",
    X_train_scaled, X_test_scaled, y_train, y_test
)
results.append(lasso_result)

# 4. Random Forest
print("\n" + "="*60)
print("4️⃣  랜덤 포레스트 (Random Forest)")
print("="*60)
print("\n📖 설명: 여러 개의 결정 트리를 결합한 앙상블 모델")
print("   비선형 관계도 잘 포착하고 과적합에 강합니다.\n")

rf_model = RandomForestRegressor(
    n_estimators=100,    # 트리 개수
    max_depth=10,        # 트리 깊이 제한 (과적합 방지)
    random_state=42,
    n_jobs=-1            # 모든 CPU 코어 사용
)
rf_result = train_and_evaluate_model(
    rf_model, "Random Forest",
    X_train, X_test, y_train, y_test  # Random Forest는 스케일링 불필요
)
results.append(rf_result)

# 5. Gradient Boosting
print("\n" + "="*60)
print("5️⃣  그래디언트 부스팅 (Gradient Boosting)")
print("="*60)
print("\n📖 설명: 순차적으로 트리를 학습하며 이전 트리의 오차를 개선")
print("   일반적으로 가장 높은 성능을 보이는 모델 중 하나입니다.\n")

gb_model = GradientBoostingRegressor(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42
)
gb_result = train_and_evaluate_model(
    gb_model, "Gradient Boosting",
    X_train, X_test, y_train, y_test
)
results.append(gb_result)

print("\n" + "="*60)
print("✅ 모든 모델 학습 완료!")
print("="*60)

In [ ]:
# ============================================================
# 5-4. 모델 성능 비교
# ============================================================

print("\n📊 모델 성능 종합 비교\n")

# 결과를 데이터프레임으로 정리
comparison_df = pd.DataFrame([
    {
        '모델': result['model_name'],
        'MAE': result['test_mae'],
        'RMSE': result['test_rmse'],
        'R² Score': result['test_r2']
    }
    for result in results
])

# 성능이 좋은 순으로 정렬 (R² Score 기준)
comparison_df = comparison_df.sort_values('R² Score', ascending=False)

print(comparison_df.to_string(index=False))

# 최고 성능 모델
best_model_name = comparison_df.iloc[0]['모델']
best_r2 = comparison_df.iloc[0]['R² Score']

print("\n" + "="*60)
print(f"🏆 최고 성능 모델: {best_model_name}")
print(f"   R² Score: {best_r2:.4f}")
print("="*60)

# 시각화
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# MAE 비교
axes[0].barh(comparison_df['모델'], comparison_df['MAE'], color='coral')
axes[0].set_xlabel('MAE (낮을수록 좋음)', fontsize=12, fontweight='bold')
axes[0].set_title('평균 절대 오차 비교', fontsize=14, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# RMSE 비교
axes[1].barh(comparison_df['모델'], comparison_df['RMSE'], color='lightblue')
axes[1].set_xlabel('RMSE (낮을수록 좋음)', fontsize=12, fontweight='bold')
axes[1].set_title('제곱근 평균 제곱 오차 비교', fontsize=14, fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)

# R² Score 비교
axes[2].barh(comparison_df['모델'], comparison_df['R² Score'], color='lightgreen')
axes[2].set_xlabel('R² Score (높을수록 좋음)', fontsize=12, fontweight='bold')
axes[2].set_title('결정계수 비교', fontsize=14, fontweight='bold')
axes[2].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*60)

## 6️⃣ 특성 중요도 분석

---

### 어떤 특성이 리뷰 유용성에 가장 큰 영향을 미칠까요?

---

In [ ]:
# ============================================================
# 6-1. Random Forest 특성 중요도
# ============================================================

print("\n📊 Random Forest 모델의 특성 중요도 분석\n")

# Random Forest 모델에서 특성 중요도 추출
rf_model_trained = [r['model'] for r in results if r['model_name'] == 'Random Forest'][0]

# 특성 중요도 추출
feature_importance = pd.DataFrame({
    '특성': available_features,
    '중요도': rf_model_trained.feature_importances_
}).sort_values('중요도', ascending=False)

print("📋 특성 중요도 순위:\n")
print(feature_importance.to_string(index=False))

# 시각화
plt.figure(figsize=(10, 8))
plt.barh(feature_importance['특성'], feature_importance['중요도'], color='steelblue')
plt.xlabel('중요도', fontsize=12, fontweight='bold')
plt.title('특성 중요도 (Random Forest)', fontsize=16, fontweight='bold', pad=20)
plt.gca().invert_yaxis()  # 중요도 높은 것이 위로
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 인사이트:")
top_3_features = feature_importance.head(3)['특성'].tolist()
print(f"\n가장 중요한 3가지 특성:")
for i, feature in enumerate(top_3_features, 1):
    importance = feature_importance[feature_importance['특성'] == feature]['중요도'].values[0]
    print(f"   {i}. {feature} (중요도: {importance:.4f})")

print("\n" + "="*60)

## 7️⃣ 예측 결과 분석 및 실전 적용

---

In [ ]:
# ============================================================
# 7-1. 실제 vs 예측 비교
# ============================================================

print("\n📊 실제 유용성 vs 예측 유용성 비교\n")

# 최고 성능 모델의 예측값 사용
best_result = results[0]  # 이미 정렬되어 있음
y_pred = best_result['predictions']

# 산점도 그리기
plt.figure(figsize=(10, 8))
plt.scatter(y_test, y_pred, alpha=0.5, s=20)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
         'r--', linewidth=2, label='완벽한 예측선')
plt.xlabel('실제 유용성', fontsize=12, fontweight='bold')
plt.ylabel('예측 유용성', fontsize=12, fontweight='bold')
plt.title(f'실제 vs 예측 유용성 ({best_result["model_name"]})', 
          fontsize=16, fontweight='bold', pad=20)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 해석:")
print("   - 점들이 빨간 선 근처에 있을수록 예측이 정확함")
print("   - 산포도가 크면 예측이 부정확함")

print("\n" + "="*60)

In [ ]:
# ============================================================
# 7-2. 새로운 리뷰의 유용성 예측 함수
# ============================================================

def predict_review_helpfulness(review_text, rating, is_recommended, 
                               product_price=None, product_rating=None):
    """
    새로운 리뷰의 유용성을 예측하는 함수
    
    Parameters:
    -----------
    review_text : str
        리뷰 텍스트
    rating : float
        평점 (1-5)
    is_recommended : bool
        추천 여부
    product_price : float, optional
        제품 가격
    product_rating : float, optional
        제품 평균 평점
    
    Returns:
    --------
    float : 예측된 유용성 점수
    """
    
    # 특성 추출
    features = {}
    
    # 텍스트 기반 특성
    features['text_length'] = len(review_text)
    features['word_count'] = len(review_text.split())
    features['sentence_count'] = len(re.findall(r'[.!?]+', review_text))
    features['avg_word_length'] = features['text_length'] / (features['word_count'] + 1)
    features['uppercase_ratio'] = sum(1 for c in review_text if c.isupper()) / (len(review_text) + 1)
    features['exclamation_count'] = review_text.count('!')
    features['question_count'] = review_text.count('?')
    
    # 리뷰 관련 특성
    features['is_recommended_num'] = int(is_recommended)
    features['has_skin_info'] = 0  # 간단히 0으로 설정
    
    # 제품 관련 특성 (제공된 경우)
    features['price_usd'] = product_price if product_price else 50.0  # 기본값
    features['rating_product'] = product_rating if product_rating else 4.0  # 기본값
    features['reviews'] = 100  # 기본값
    features['rating_diff'] = abs(rating - features['rating_product'])
    
    # 옵션 특성들 (있으면 추가)
    if 'has_title' in available_features:
        features['has_title'] = 0
        features['title_length'] = 0
    
    if 'has_skin_tone' in available_features:
        features['has_skin_tone'] = 0
    
    # DataFrame으로 변환 (모델 입력 형식)
    feature_df = pd.DataFrame([features])[available_features]
    
    # 예측 (Random Forest 사용)
    prediction = rf_model_trained.predict(feature_df)[0]
    
    return prediction

print("✅ 리뷰 유용성 예측 함수 생성 완료!")
print("="*60)

In [ ]:
# ============================================================
# 🎮 내 리뷰의 유용성 예측해보기
# ============================================================

print("\n" + "="*60)
print("🎮 리뷰 유용성 예측 테스트")
print("="*60 + "\n")

# 예시 리뷰 1: 짧고 간단한 리뷰
review1 = "Great product!"
pred1 = predict_review_helpfulness(
    review_text=review1,
    rating=5.0,
    is_recommended=True
)

print("\n📝 예시 1: 짧은 리뷰")
print(f"   리뷰: \"{review1}\"")
print(f"   평점: 5.0 | 추천: Yes")
print(f"   🎯 예측 유용성: {pred1:.2f}\n")

# 예시 리뷰 2: 자세하고 구체적인 리뷰
review2 = """I have combination skin and this moisturizer works perfectly for me. 
It's lightweight but very hydrating. I've been using it for 3 months and noticed 
my skin is much smoother. The price is a bit high but a little goes a long way. 
Would definitely recommend for people with similar skin type."""

pred2 = predict_review_helpfulness(
    review_text=review2,
    rating=4.5,
    is_recommended=True,
    product_price=65.0,
    product_rating=4.3
)

print("\n📝 예시 2: 자세한 리뷰")
print(f"   리뷰: \"{review2[:100]}...\"")
print(f"   평점: 4.5 | 추천: Yes")
print(f"   제품 가격: $65.00 | 제품 평점: 4.3")
print(f"   🎯 예측 유용성: {pred2:.2f}\n")

print("\n" + "="*60)
print("💡 결론")
print("="*60)
print(f"\n예시 1 (짧은 리뷰) 유용성: {pred1:.2f}")
print(f"예시 2 (자세한 리뷰) 유용성: {pred2:.2f}")
print(f"\n차이: {abs(pred2 - pred1):.2f} → 자세한 리뷰가 약 {(pred2/pred1):.1f}배 더 유용!\n")

print("="*60)

## 8️⃣ 좋은 리뷰 작성 가이드라인

---

### 📝 분석 결과를 바탕으로 한 실전 가이드

---

In [ ]:
# ============================================================
# 8-1. 좋은 리뷰 vs 나쁜 리뷰 비교
# ============================================================

print("\n" + "="*60)
print("📚 유용한 리뷰 작성 가이드라인")
print("="*60 + "\n")

print("✅ 유용한 리뷰의 특징:\n")
print("   1. 적절한 길이 (100-500자, 약 2-10문장)")
print("      - 너무 짧지도, 너무 길지도 않게")
print("      - 핵심 정보를 명확하게 전달\n")

print("   2. 구체적인 정보 포함")
print("      - 피부 타입/피부 톤 명시")
print("      - 사용 기간 언급")
print("      - 구체적인 효과 설명\n")

print("   3. 장단점 균형")
print("      - 좋은 점과 아쉬운 점 모두 언급")
print("      - 극단적인 평점보다 중간 평점이 더 신뢰\n")

print("   4. 객관적인 표현")
print("      - 과도한 감탄사(!!!) 자제")
print("      - 대문자 남용 피하기")
print("      - 사실 기반 설명\n")

print("   5. 비교 정보")
print("      - 유사 제품과의 비교")
print("      - 가성비 언급")
print("      - 어떤 사람에게 추천하는지 명시\n")

print("-" * 60 + "\n")

print("❌ 유용하지 않은 리뷰의 특징:\n")
print("   1. 너무 짧음")
print("      - \"좋아요!\", \"별로예요\" 같은 한 마디\n")

print("   2. 감정만 표현")
print("      - 구체적인 이유 없이 \"최고!!!\", \"최악...\"\n")

print("   3. 제품과 무관한 내용")
print("      - 배송 불만, 포장 문제 등\n")

print("   4. 개인 정보 없음")
print("      - 피부 타입, 사용 기간 등 맥락 부족\n")

print("="*60)

In [ ]:
# ============================================================
# 8-2. 리뷰 작성 템플릿
# ============================================================

print("\n" + "="*60)
print("📝 유용한 리뷰 작성 템플릿")
print("="*60 + "\n")

template = """
【 기본 정보 】
- 피부 타입: [건성/지성/복합성/중성]
- 피부 고민: [예: 건조함, 여드름, 주름 등]
- 사용 기간: [예: 2주, 1개월, 3개월 등]

【 제품 평가 】
1. 좋았던 점:
   - [구체적인 효과 1]
   - [구체적인 효과 2]

2. 아쉬운 점:
   - [단점이나 개선점]

3. 사용 팁:
   - [사용 방법이나 꿀팁]

【 추천 여부 】
- 추천 대상: [어떤 사람에게 추천하는지]
- 비추천 대상: [어떤 사람에게는 안 맞을지]
- 가성비: [가격 대비 만족도]

【 종합 의견 】
[한 문장으로 요약]
"""

print(template)

print("\n" + "="*60)
print("💡 이 템플릿을 따라 작성하면 유용성 점수가 높을 것입니다!")
print("="*60)

## 9️⃣ 요약 및 결론

---

### 🎓 이번 노트북에서 배운 내용

1. **리뷰 유용성의 중요성**
   - 좋은 리뷰는 다른 사용자의 구매 결정에 큰 도움
   - 유용성을 예측하여 좋은 리뷰를 상단에 노출

2. **특성 공학 (Feature Engineering)**
   - 원본 데이터에서 15개 이상의 특성 추출
   - 텍스트 길이, 단어 수, 문장 수, 평점, 가격 등

3. **머신러닝 모델 학습**
   - 5가지 회귀 모델 학습 및 비교
   - Random Forest와 Gradient Boosting이 좋은 성능

4. **특성 중요도 분석**
   - 리뷰 길이, 단어 수, 제품 인기도가 중요
   - 평점과 추천 여부도 영향

5. **실전 적용**
   - 새로운 리뷰의 유용성 예측 함수 구현
   - 좋은 리뷰 작성 가이드라인 도출

---

### 💡 주요 인사이트

1. **적절한 리뷰 길이**: 100-500자 (2-10문장)가 가장 유용
2. **균형잡힌 평가**: 극단적 평점보다 중간 평점이 더 신뢰
3. **구체적인 정보**: 피부 타입, 사용 기간, 효과를 명시
4. **객관적 표현**: 과도한 감탄사나 대문자 자제

---

### 🚀 프로젝트 완료!

축하합니다! 6개의 노트북을 모두 완료했습니다:

1. ✅ 01_data_preprocessing.ipynb - 데이터 전처리
2. ✅ 02_exploratory_analysis.ipynb - 탐색적 분석
3. ✅ 03_ml_rating_prediction.ipynb - 평점 예측 (회귀)
4. ✅ 04_ml_sentiment_analysis.ipynb - 감성 분석 (분류)
5. ✅ 05_ml_recommendation_system.ipynb - 추천 시스템
6. ✅ 06_ml_review_helpfulness.ipynb - 리뷰 유용성 예측

---

### 📚 배운 기술 스택

- **데이터 처리**: pandas, numpy
- **시각화**: matplotlib, seaborn
- **머신러닝**: scikit-learn, xgboost
- **텍스트 분석**: NLTK, TF-IDF
- **회귀**: Linear Regression, Ridge, Lasso, Random Forest, Gradient Boosting
- **분류**: Logistic Regression, Naive Bayes, SVM, Random Forest
- **추천 시스템**: 코사인 유사도, 콘텐츠 기반 필터링

---

### ✅ 체크리스트

- [ ] 리뷰 유용성의 개념을 이해했나요?
- [ ] 특성 공학의 중요성을 이해했나요?
- [ ] 5가지 회귀 모델을 모두 실행해봤나요?
- [ ] 특성 중요도를 확인했나요?
- [ ] 새로운 리뷰의 유용성을 예측해봤나요?
- [ ] 좋은 리뷰 작성 가이드라인을 이해했나요?

---

**정말 수고하셨습니다! 🎉🎊**

이제 여러분은 실전 데이터 과학 프로젝트를 완수했습니다!

배운 내용을 바탕으로 다른 데이터셋에도 적용해보세요! 💪